In [2]:
import pandas as pd
import numpy as np
import itertools
import os
import re
import requests
import json

# Gather

In [4]:
# pull standard cards from mtgjson
standard_sets = requests.get('https://www.mtgjson.com/files/StandardPrintings.json')
standard_sets_clean = standard_sets.json()

# code to save standard_sets as a json file 
with open('standard_sets_clean.txt', 'w') as outfile:
    json.dump(standard_sets_clean, outfile)

# code to load standard_sets_clean
with open('standard_sets_clean.txt', encoding='utf-8') as file:
    standard_sets_clean = json.load(file)

In [2]:
decks = pd.read_csv('C:/Users/muroc/Documents/MTG/data/decks.csv')

FileNotFoundError: [Errno 2] File b'C:/Users/muroc/Documents/MTG/data/decks.csv' does not exist: b'C:/Users/muroc/Documents/MTG/data/decks.csv'

In [42]:
decks = pd.read_csv('~/Documents/programs/python/MTGA_rarity_analysis/data/mtg_data.csv')

In [43]:
decks_clean = decks.copy()

In [32]:
decks_clean = pd.read_csv('~/Documents/MTGA_rarity_analysis/data/decks_clean.csv')

# Assess

In [33]:
decks_clean.head()

,player,url,deck_name,num,card_name,card_rarity
0,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,2,Forest,common
1,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,4,Shock,common
2,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,3,Forest,common
3,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,1,Island,common
4,Vincent_Choy,https://www.mtgtop8.com/event?e=23600&d=363526...,5c_PW,1,Island,common


In [53]:
decks_clean.tail()

,player,url,deck_name,num,card_name
11046,Chingpai,https://www.mtgtop8.com/event?e=23587&d=363362...,Jeskai_Planeswalker,1,Blast Zone
11047,Fulgence,https://www.mtgtop8.com/event?e=23856&d=365902...,Niv-Mizzet_Reborn,1,"Guild Globe,"
11048,Chingpai,https://www.mtgtop8.com/event?e=23678&d=364322...,Jeskai_Planeswalker,2,"Interplanar Beacon,"
11049,Chingpai,https://www.mtgtop8.com/event?e=23622&d=363723...,Jeskai_Planeswalker,2,"Interplanar Beacon,"
11050,Chingpai,https://www.mtgtop8.com/event?e=23587&d=363362...,Jeskai_Planeswalker,2,"Interplanar Beacon,"


In [55]:
decks_clean.sample(10)

,player,url,deck_name,num,card_name
10181,Dylan_Gottesman,https://www.mtgtop8.com/event?e=24059&d=367919...,Jund_Sacrifice,4,Casualties of War
994,Kurt_Stockbrugger,https://www.mtgtop8.com/event?e=23600&d=363527...,Esper_DOOM,2,Swamp
4980,Maesono_Kanta,https://www.mtgtop8.com/event?e=23968&d=367033...,Adventure,4,Escape to the Wilds
4763,Erikhov_Maxim,https://www.mtgtop8.com/event?e=23741&d=364838...,Mono-red_Aggro,4,Runaway Steam-Kin
3875,Hang_Sheng_Kwang,https://www.mtgtop8.com/event?e=23574&d=363266...,Mono_Black,3,Midnight Reaper
9925,Chris_Yap,https://www.mtgtop8.com/event?e=23704&d=364532...,Selesnya_Adventure,3,Gideon Blackblade
9604,Julian_John,https://www.mtgtop8.com/event?e=24054&d=367877...,Simic_Flash,3,Paradise Druid
10317,Jeffery_Mullins,https://www.mtgtop8.com/event?e=23993&d=367256...,Adventure,1,Planewide Celebration
5545,Venom1,https://www.mtgtop8.com/event?e=23925&d=366567...,Golgari_Aggro,4,Midnight Reaper
1581,Peter_Yong,https://www.mtgtop8.com/event?e=24055&d=367885...,Jund_Sacrifice,4,Blood Crypt


In [56]:
decks_clean

,player,url,deck_name,num,card_name
0,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,2,Forest
1,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,4,Shock
2,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,3,Forest
3,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,1,Island
4,Vincent_Choy,https://www.mtgtop8.com/event?e=23600&d=363526...,5c_PW,1,Island
...,...,...,...,...,...
11046,Chingpai,https://www.mtgtop8.com/event?e=23587&d=363362...,Jeskai_Planeswalker,1,Blast Zone
11047,Fulgence,https://www.mtgtop8.com/event?e=23856&d=365902...,Niv-Mizzet_Reborn,1,"Guild Globe,"
11048,Chingpai,https://www.mtgtop8.com/event?e=23678&d=364322...,Jeskai_Planeswalker,2,"Interplanar Beacon,"
11049,Chingpai,https://www.mtgtop8.com/event?e=23622&d=363723...,Jeskai_Planeswalker,2,"Interplanar Beacon,"


In [64]:
decks

,Unnamed: 0,player,url,name,cards
0,0,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,2 Forest
1,1,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,4 Shock
2,2,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,3 Forest
3,3,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,1 Island
4,4,Vincent_Choy,https://www.mtgtop8.com/event?e=23600&d=363526...,5c_PW,1 Island
...,...,...,...,...,...
11046,17341,Chingpai,https://www.mtgtop8.com/event?e=23587&d=363362...,Jeskai_Planeswalker,1 Blast Zone
11047,17416,Fulgence,https://www.mtgtop8.com/event?e=23856&d=365902...,Niv-Mizzet_Reborn,"1 Guild Globe,"
11048,17790,Chingpai,https://www.mtgtop8.com/event?e=23678&d=364322...,Jeskai_Planeswalker,"2 Interplanar Beacon,"
11049,17791,Chingpai,https://www.mtgtop8.com/event?e=23622&d=363723...,Jeskai_Planeswalker,"2 Interplanar Beacon,"


## Assessment
- [done] deck_list column has , for delimiter but names can also have a comma
- [done] cards are in one column
- [done] multiple cards are in one object
- [done] card number and card name are the same object
- [done] null values in `deck_list` column
- [done] rarities not present in the data
- [done] some card names still have a comma at the end
- some decks have multiple observations
- many cards have nan values for rarity
- dual cards are considered two cards in standard_sets_clean but only one card in decks_clean

# Clean

## Define
- filter for any lists in `deck_list` that have null values in it

## Code

In [11]:
decks_clean.deck_list = decks_clean[decks_clean.astype(str)['deck_list'] != "['nan']"]

## Test

In [12]:
decks_clean[decks_clean.astype(str)['deck_list'] == "['nan']"]

,deck_list,player,url,name


## Define
- make , in `deck_list` into '|' without mistaking commas in names

## Code

In [14]:
decks_clean.deck_list = decks_clean.deck_list.str.replace(', (?=\d)', '|')

## Define
- split `deck_list` by card

## Code

In [15]:
new = decks_clean.deck_list.str.split('|', expand=True)
decks_clean = decks_clean.join(new)

## Test

In [16]:
decks_clean.head()

,deck_list,player,url,name,0,1,2,3,4,5,...,30,31,32,33,34,35,36,37,38,39
0,2 Forest|1 Island|1 Mountain|2 Swamp|3 Breedin...,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,2 Forest,1 Island,1 Mountain,2 Swamp,3 Breeding Pool,2 Mystical Dispute,...,None,None,None,None,None,None,None,None,None,None
1,4 Shock|1 Forest|1 Island|1 Mountain|1 Swamp|1...,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,4 Shock,1 Forest,1 Island,1 Mountain,1 Swamp,1 Blood Crypt,...,2 Overgrown Tomb,2 Watery Grave,1 Temple of Mystery,"3 Neoform,",None,None,None,None,None,None
2,3 Forest|1 Island|2 Plains|1 Swamp|4 Breeding ...,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,3 Forest,1 Island,2 Plains,1 Swamp,4 Breeding Pool,4 Gilded Goose,...,None,None,None,None,None,None,None,None,None,None
3,1 Island|1 Mountain|2 Plains|3 Swamp|2 Blood C...,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,1 Island,1 Mountain,2 Plains,3 Swamp,2 Blood Crypt,1 Hallowed Fountain,...,4 Oath of Kaya,4 Guild Globe,"2 Sorcerous Spyglass,",None,None,None,None,None,None,None
4,1 Island|1 Mountain|1 Plains|4 Hallowed Founta...,Vincent_Choy,https://www.mtgtop8.com/event?e=23600&d=363526...,5c_PW,1 Island,1 Mountain,1 Plains,4 Hallowed Fountain,3 Fae of Wishes,4 Fires of Invention,...,"4 Teferi, Time Raveler",1 Time Wipe,"1 Ashiok, Dream Render",1 Blast Zone,4 Interplanar Beacon,1 Karn's Bastion,"1 Sorcerous Spyglass,",None,None,None


## Define 
- melt cards into one column

## Code

In [17]:
decks_clean = pd.melt(decks_clean, id_vars=['deck_list', 'player', 'url', 'name'], value_vars=decks_clean.columns[4:45])

In [18]:
decks_clean = decks_clean.rename(columns={'value':'cards'}).drop(columns=['deck_list', 'variable'])

## Test

In [19]:
decks_clean.shape

(18040, 4)

In [20]:
decks_clean.head()

,player,url,name,cards
0,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,2 Forest
1,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,4 Shock
2,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,3 Forest
3,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,1 Island
4,Vincent_Choy,https://www.mtgtop8.com/event?e=23600&d=363526...,5c_PW,1 Island


## Define
- drop rows that have 'None' in them

## Code

In [21]:
decks_clean.cards = decks_clean.cards.astype('str')

In [22]:
decks_clean = decks_clean[decks_clean.cards != 'None']

In [46]:
decks_clean.to_csv('~/Documents/programs/python/MTGA_rarity_analysis/data/decks_clean.csv', index=False)

## Test

In [65]:
decks_clean[decks_clean.cards in 'None']

AttributeError: 'DataFrame' object has no attribute 'cards'

## Define
- get numbers out of `cards` column

## Code

In [48]:
new = decks_clean.cards.str.split('(?<=\d)\s', expand=True)
decks_clean = decks_clean.join(new).rename(columns={'name':'deck_name', 0:'num', 1:'card_name'}).drop(columns=['cards'])

## Test

In [49]:
decks_clean.head()

,player,url,deck_name,num,card_name
0,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,2,Forest
1,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,4,Shock
2,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,3,Forest
3,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,1,Island
4,Vincent_Choy,https://www.mtgtop8.com/event?e=23600&d=363526...,5c_PW,1,Island


In [50]:
decks_clean.to_csv('~/Documents/programs/python/MTGA_rarity_analysis/data/decks_clean.csv', index=False)

## Define
- Add card rarity to decks_clean

In [11]:
# code to get rarity in a list by themselves
card_rarities = []
for i in standard_sets_clean.keys():
    current_set = standard_sets_clean[i]['cards']
    for a in np.arange(np.count_nonzero(current_set)):
        rarity = current_set[a]['rarity']
        card_rarities.append(rarity)

In [12]:
# code to get names in a list by themselves
card_names = []
for i in standard_sets_clean.keys():
    current_set = standard_sets_clean[i]['cards']
    for a in np.arange(np.count_nonzero(current_set)):
        name = current_set[a]['name']
        card_names.append(name)

In [13]:
# code to create dataframe of card_rarities and card_names
mtg_card_rarities = pd.DataFrame({'card_name': card_names, 'card_rarity': card_rarities})
mtg_card_rarities_clean = mtg_card_rarities.copy()
mtg_card_rarities_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/mtg_card_rarities_clean.csv', index=False)

In [14]:
# drop duplicates in mtg_card_rarities_clean
mtg_card_rarities_clean.drop_duplicates(inplace=True)

In [15]:
mtg_card_rarities_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/mtg_card_rarities_clean.csv', index=False)

In [16]:
mtg_card_rarities_clean = pd.read_csv('~/Documents/MTGA_rarity_analysis/data/mtg_card_rarities_clean.csv')

In [19]:
decks_clean = decks_clean.join(mtg_card_rarities_clean.set_index('card_name'), on='card_name')

ValueError: columns overlap but no suffix specified: Index(['card_rarity'], dtype='object')

## Test

In [18]:
mtg_card_rarities_clean

,card_name,card_rarity
0,Acclaimed Contender,rare
1,"Alela, Artful Provocateur",mythic
2,All That Glitters,uncommon
3,Bring to Life,uncommon
4,Animating Faerie,uncommon
...,...,...
1415,Wall of Runes,common
1416,Wanderer's Strike,common
1417,War Screecher,common
1418,Wardscale Crocodile,common


In [20]:
decks_clean

,player,url,deck_name,num,card_name,card_rarity
0,Julian_Felix_Flury,https://www.mtgtop8.com/event?e=23581&d=363309...,4c_Control,2,Forest,common
1,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,4,Shock,common
2,Keylan_Steffins,https://www.mtgtop8.com/event?e=23516&d=362603...,4c_Value,3,Forest,common
3,Wynand_Van_Zyl,https://www.mtgtop8.com/event?e=23773&d=365167...,4_Color_Doom_,1,Island,common
4,Vincent_Choy,https://www.mtgtop8.com/event?e=23600&d=363526...,5c_PW,1,Island,common
...,...,...,...,...,...,...
11046,Chingpai,https://www.mtgtop8.com/event?e=23587&d=363362...,Jeskai_Planeswalker,1,Blast Zone,rare
11047,Fulgence,https://www.mtgtop8.com/event?e=23856&d=365902...,Niv-Mizzet_Reborn,1,Guild Globe,NaN
11048,Chingpai,https://www.mtgtop8.com/event?e=23678&d=364322...,Jeskai_Planeswalker,2,Interplanar Beacon,NaN
11049,Chingpai,https://www.mtgtop8.com/event?e=23622&d=363723...,Jeskai_Planeswalker,2,Interplanar Beacon,NaN


In [40]:
decks_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11051 entries, 0 to 11050
Data columns (total 6 columns):
player         11051 non-null object
url            11051 non-null object
deck_name      11028 non-null object
num            11051 non-null int64
card_name      11051 non-null object
card_rarity    10519 non-null object
dtypes: int64(1), object(5)
memory usage: 518.1+ KB


In [41]:
type(decks_clean.card_rarity[0])

str

In [42]:
decks_clean.sample(10)

,player,url,deck_name,num,card_name,card_rarity
1621,Mh_voltage,https://www.mtgtop8.com/event?e=23874&d=366097...,Mono_Black,4,Murderous Rider,rare
194,William_Jensen,https://www.mtgtop8.com/event?e=23942&d=366834...,Izzet_Flash,9,Island,common
2908,Chingpai,https://www.mtgtop8.com/event?e=23622&d=363723...,Jeskai_Planeswalker,1,Realm-Cloaked Giant,mythic
7720,_Tia93_,https://www.mtgtop8.com/event?e=23594&d=363483...,Bant_Control,3,Growth Spiral,common
4570,Nebykov_Denis,https://www.mtgtop8.com/event?e=23741&d=364835...,Bant_Food,4,Fabled Passage,rare
6082,Edgar_Magalhaes,https://www.mtgtop8.com/event?e=24032&d=367629...,Jund_Sacrifice,3,Lovestruck Beast,rare
6353,Brener,https://www.mtgtop8.com/event?e=23590&d=363420...,Bant_Control,3,Temple Garden,rare
3515,Leonardo_Martins,https://www.mtgtop8.com/event?e=23974&d=367072...,Simic_Ramp,1,Castle Vantress,rare
10472,Yixian_Liang,https://www.mtgtop8.com/event?e=23503&d=362414...,Sultai_Food,4,Paradise Druid,uncommon
3780,Robert_Berni,https://www.mtgtop8.com/event?e=24055&d=367889...,Izzet_Control,3,Fabled Passage,rare


In [57]:
decks_clean.card_name.nunique(), dc_rarity_null.card_name.nunique()

(466, 66)

In [58]:
mtg_card_rarities_clean.shape

(1420, 2)

In [52]:
dc_rarity_null = decks_clean[decks_clean.card_rarity.isna()]

In [55]:
dc_rarity_null.card_name.value_counts()

Sorcerous Spyglass,                90
Mayhem Devil,                      34
Casualties of War,                 30
Expansion-Explosion                28
Tolsimir, Friend to Wolves,        26
                                   ..
Call of the Conclave                1
Angrath, Captain of Chaos,          1
Massacre                            1
Tomik, Distinguished Advokist,      1
Carnival-Carnage                    1
Name: card_name, Length: 66, dtype: int64

In [59]:
decks_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/decks_clean.csv', index=False)

## Define
- get rid of nan in `card_name`

## Code 

In [7]:
decks_clean.card_name = decks_clean.card_name.str.replace(', $', '')

In [8]:
decks_clean = decks_clean.join(mtg_card_rarities_clean.set_index('card_name'), on='card_name', lsuffix='x').drop(columns='card_rarityx')

NameError: name 'mtg_card_rarities_clean' is not defined

In [9]:
dual_cards = decks_clean[decks_clean.card_name.str.contains('.+-.+') & decks_clean.card_rarity.isna()]
new = dual_cards.card_name.str.split('-', expand=True)
dual_cards = dual_cards.join(new)

In [10]:
dual_cards = dual_cards.melt(id_vars=list(decks_clean.columns), value_vars=[0, 1], value_name='card_name_2').drop(columns=['variable'])

In [11]:
decks_clean = decks_clean.join(dual_cards[['card_name', 'card_name_2']].set_index('card_name'), on='card_name')

In [88]:
decks_clean.card_rarity = decks_clean.card_rarity.astype('str')
for i in np.arange(decks_clean.shape[0]):
    if decks_clean.card_name[i] == 'Expansion-Explosion':
        re.sub('', 'rare', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Collision-Colossus':
        re.sub('', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Flower-Flourish':
        re.sub('', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Thrash-Threat':
        re.sub('', 'rare', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Discovery-Dispersal':
        re.sub('', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Stasus-Statue':
        re.sub('', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Bedeck-Bedazzle':
        re.sub('', 'rare', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Consecrate-Consume':
        re.sub('', 'uncommon', decks_clean.card_rarity[i])
    elif decks_clean.card_name[i] == 'Carnival-Carnage':
        re.sub('', 'uncommon', decks_clean.card_rarity[i])

In [89]:
decks_clean.card_rarity.dropna(inplace=True)

## Test

In [90]:
decks_clean.card_rarity.isna().sum()

0

In [34]:
dc_rarity_null = decks_clean[decks_clean.card_rarity.isna()]

In [84]:
dc_rarity_null.shape[0]

80

In [82]:
dc_rarity_null.card_name.value_counts()

Expansion-Explosion     28
Collision-Colossus      13
Flower-Flourish         12
Find-Finality           12
Thrash-Threat            3
Unknown Card             2
Discovery-Dispersal      2
Massacre                 1
Silhana Ledgewalker      1
Call of the Conclave     1
Status-Statue            1
Angrath's Fury           1
Bedeck-Bedazzle          1
Consecrate-Consume       1
Carnival-Carnage         1
Name: card_name, dtype: int64

In [89]:
standard_sets_clean.keys()

dict_keys(['ELD', 'GRN', 'M20', 'RNA', 'WAR'])

In [94]:
names = []
for i in standard_sets_clean['GRN']['cards']:
    names.append(i['name'])

In [95]:
names

['Affectionate Indrik',
 'Arboretum Elemental',
 'Arclight Phoenix',
 'Artful Takedown',
 "Assassin's Trophy",
 'Assemble',
 'Assure',
 'Attendant of Vraska',
 'Aurelia, Exemplar of Justice',
 'Barging Sergeant',
 'Barrier of Bones',
 'Bartizan Bats',
 'Beacon Bolt',
 'Beamsplitter Mage',
 'Beast Whisperer',
 'Blade Instructor',
 'Blood Operative',
 'Book Devourer',
 'Boros Challenger',
 'Boros Guildgate',
 'Boros Guildgate',
 'Boros Locket',
 'Bounty Agent',
 'Bounty of Might',
 'Burglar Rat',
 'Camaraderie',
 'Candlelight Vigil',
 'Capture Sphere',
 'Centaur Peacemaker',
 'Chamber Sentry',
 'Chance for Glory',
 'Charnel Troll',
 "Chemister's Insight",
 'Child of Night',
 'Chromatic Lantern',
 'Circuitous Route',
 'Citywatch Sphinx',
 'Citywide Bust',
 'Collar the Culprit',
 'Command the Storm',
 'Conclave Cavalier',
 'Conclave Guildmage',
 'Conclave Tribunal',
 'Concoct',
 'Connive',
 'Cosmotronic Wave',
 'Crackling Drake',
 'Creeping Chill',
 'Crush Contraband',
 'Crushing Canopy',


In [96]:
decks_clean[decks_clean.card_name == 'Massacre']

,player,url,deck_name,num,card_name,card_rarity
7898,David_Pound,https://www.mtgtop8.com/event?e=23943&d=366838...,Lucky_Golgari,1,Massacre,NaN


In [108]:
dual_cards.head()

,player,url,deck_name,num,card_name,card_rarity,0,1
3541,Rémi_D'emmerez,https://www.mtgtop8.com/event?e=23754&d=364990...,Sultai_Oko-less,2,Find-Finality,NaN,Find,Finality
4460,Nobuta_Kazuhito,https://www.mtgtop8.com/event?e=23980&d=367109...,Temur_Aggro,1,Status-Statue,NaN,Status,Statue
4511,Robert_Lippmann,https://www.mtgtop8.com/event?e=23476&d=362227...,4c_Elementals,2,Find-Finality,NaN,Find,Finality
4919,Bullwinkkle6705,https://www.mtgtop8.com/event?e=23653&d=364109...,Temur_Reclamation,3,Expansion-Explosion,NaN,Expansion,Explosion
4923,Hibino_Yasutaka,https://www.mtgtop8.com/event?e=23749&d=364939...,Temur_Reclamation,4,Expansion-Explosion,NaN,Expansion,Explosion


In [109]:
dual_cards[0]

3541          Find
4460        Status
4511          Find
4919     Expansion
4923     Expansion
           ...    
8822        Flower
10441    Collision
10442    Collision
10608       Thrash
10609       Thrash
Name: 0, Length: 74, dtype: object

In [111]:
list(decks_clean.columns)

['player', 'url', 'deck_name', 'num', 'card_name', 'card_rarity']

In [91]:
decks_clean.to_csv('~/Documents/MTGA_rarity_analysis/data/decks_clean.csv', index=False)